#### Microsoft Azure open data program https://azure.microsoft.com/en-us/services/open-datasets/catalog/naip/

#### This code utilizes the capability of Cloud Optimized Geotiffs to analyze NAIP tiles covering a large area of interest. Instead of downloading each tile manually to use it later for analysis, it can be streamed directly from an Azure blob. 
#### The analysis (in this case NDVI or MSAVI classification) can be completed and saved for later use, while also saving limited drive space available to me.

In [1]:
import pandas as pd
import geopandas as gpd
import os, shutil
import rasterio
import urllib
import re
import fiona.transform
import requests
import numpy as np
import gdal 
gdal.SetConfigOption("GDAL_HTTP_UNSAFESSL", "YES")
gdal.VSICurlClearCache()

In [3]:
# Put the quad and aoi files in the infilepath directory, define a different outpath if desired
infilepath = r'D:\code\NAIP download'
outpath = infilepath

#text to append to each NAIP quad's complete analysis
outfilen_append = '_ndvi.tif'

# shapefile from microsoft Azure with NAIP quads
# https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_shpfl_2018/index.html
inNAIPquad = 'NAIP_18_CA.shp'

#file with polygons/points of interest. If using the whole state: aoi_file = 'centroid' (to get the centroid of each scene)
aoi_file = 'CA_aoi.gpkg'
#aoi_file = 'centroid'

In [4]:
# NAIP metadata
year = '2018' 
state = 'ca' #lowercase
resolution = '060cm'
blob_root = 'https://naipeuwest.blob.core.windows.net/naip'

In [5]:
## Read the aoi file. if using the whole state, the code will create a list of tiles based on centroids later
if aoi_file == 'centroid':
    aoi = 'centroid'
else:
    aoi = gpd.read_file(os.path.join(infilepath, aoi_file))

In [6]:
def createListofTiles(inpath,NAIPquad,inaoi):
    # NAIP Quad file
    ntiles = gpd.GeoDataFrame.from_file(os.path.join(inpath,NAIPquad))
    # if inaoi == 'centroid':
    if isinstance(inaoi, str):
        print('centroid')
        aoicreate = gpd.GeoDataFrame(ntiles.geometry.centroid, columns=['geometry'])
        inaoi = gpd.sjoin(aoicreate, ntiles, how="inner", op='intersects')
        inaoi = inaoi[['geometry']]    
    
    if inaoi.crs != ntiles.crs:
        print(inaoi.crs,ntiles.crs)
        c = ntiles.crs.dtypes
        print(c)
        inaoi= inaoi.to_crs(c)
    
    
    # sort so that the images with the most aoi's get processed first
    if str((aoi.geom_type == 'Point')[0]) == 'True':
        # Intersection with AOI to the names of all the tiles needed
        quad_intersection = gpd.sjoin(ntiles, inaoi, how="right", op='intersects')
        print(quad_intersection)
        
        print('Sorting by counts of points in each quad')
        quad_intersection['counts'] = quad_intersection.groupby(['FileName'])['FileName'].transform('count')
        quad_intersection = quad_intersection.sort_values(by='counts',ascending=False)
    else:
        # Intersection with AOI to the names of all the tiles needed
        quad_intersection = gpd.sjoin(inaoi, ntiles, how="left", op='intersects')
        print(quad_intersection)
        
    quadlist = quad_intersection.FileName.unique()
    return quadlist

In [7]:
ql = createListofTiles(infilepath,inNAIPquad,aoi)

                ID  FEATTYP ORDER00 ORDER01 ORDER02 ORDER03 ORDER04 ORDER05  \
0   68400000000048     1119     USA      CA    None    None    None    None   
0   68400000000048     1119     USA      CA    None    None    None    None   
0   68400000000048     1119     USA      CA    None    None    None    None   
0   68400000000048     1119     USA      CA    None    None    None    None   
0   68400000000048     1119     USA      CA    None    None    None    None   
0   68400000000048     1119     USA      CA    None    None    None    None   
0   68400000000048     1119     USA      CA    None    None    None    None   
0   68400000000048     1119     USA      CA    None    None    None    None   
0   68400000000048     1119     USA      CA    None    None    None    None   
0   68400000000048     1119     USA      CA    None    None    None    None   
0   68400000000048     1119     USA      CA    None    None    None    None   
0   68400000000048     1119     USA      CA    None 

c:\python27v2\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in find_intersects (vectorized)
  outputs = ufunc(*inputs)


In [8]:
# Selected indexes for analysis
def calcNDVI(src):
    # read red and nir bands, change to dtype to float64 for output ndvi calc
    red,nir = src.read(1).astype('float64'),src.read(4).astype('float64')
    
    # Calculate NDVI
    print('Calculate NDVI')
    ndvi = np.where((nir+red) == 0, 0, (nir-red)/(nir+red))
    return ndvi
    
    
def calcClassedNDVI(src):
    # read red and nir bands, change to dtype to float64 for output ndvi calc
    red,nir = src.read(1).astype('float64'),src.read(4).astype('float64')
    
    # Calculate NDVI
    print('Calculate NDVI')
    ndvi = np.where((nir+red) == 0, 0, (nir-red)/(nir+red))

    # Return reclassified NDVI into fuel (1) or no fuel (0)
    # Change NDVI thresholds where applicable
    ndvi[ndvi>.2] = 1
    ndvi[ndvi<=.2] = 0
    return ndvi


def calcMSAVI2(src):
    from math import sqrt
    # read red and nir bands, change to dtype to float64 for output calc
    red,nir = src.read(1).astype('float64'),src.read(4).astype('float64')
    
    print('Calculate MSAVI2')
    msavi2 = (2 * (nir.astype(float) + 1) - np.sqrt((2 * nir.astype(float) + 1)**2 - 8 * (nir.astype(float) - red.astype(float))))/2
    return msavi2
    
    

In [9]:
for n in range(len(ql)):
    for filen in (ql):
        print(str(n) + ' of ' + str(len(ql)))

        # Retrieve the name of the quad file and define link to file
        quadrangle = filen[2:7]
        filename = filen
        tile_url = blob_root + '/v002/' + state + '/' + year + '/' + state + '_' + resolution + \
        '_' + year + '/' + quadrangle + '/' + filename

        print(tile_url)

        # Output file name
        outputanalysis = os.path.join(outpath,filename[:-4]+outfilen_append)

        # If the file has not already been completed previously- stream the data, perform the calculation, and save output
        if not os.path.exists(outputanalysis):
            with rasterio.open(tile_url) as f:

                #### Update analysis to be run
                analysis = calcNDVI(f)

                # set transform
                affine= f.transform


                with rasterio.open(outputanalysis, 'w', 
                                           driver='GTiff',
                                           height=f.shape[0],
                                           width=f.shape[1],
                                           compress='lzw', #lossless
                                           count=1,
                                           dtype= 'float64', # 'uint8' for scaled ndvi or reclass
                                           crs= f.crs,
                                           transform=affine) as dst:
                                               dst.write(analysis,1)
                print(filename[:-4]+ outfilen_append + ' Complete')

1 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37121/m_3712155_nw_10_060_20180714_20190209.tif
2 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37121/m_3712155_ne_10_060_20180714_20190209.tif
3 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37121/m_3712154_nw_10_060_20180714_20190210.tif
4 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37121/m_3712154_ne_10_060_20180714_20190210.tif
5 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37121/m_3712153_ne_10_060_20180723_20190210.tif
6 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37121/m_3712156_nw_10_060_20180714_20190209.tif
7 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37121/m_3712153_nw_10_060_20180725_20190210.tif
8 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/

74 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712026_se_10_060_20180715_20190210.tif
75 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712025_se_10_060_20180715_20190209.tif
76 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712027_se_10_060_20180715_20190210.tif
77 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712027_ne_10_060_20180715_20190210.tif
78 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712028_ne_10_060_20180724_20190210.tif
79 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712027_nw_10_060_20180715_20190210.tif
80 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712025_ne_10_060_20180715_20190209.tif
81 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060

156 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/41123/m_4112306_nw_10_060_20181011_20190209.tif
157 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/41124/m_4112407_nw_10_060_20181007_20190210.tif
158 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/41123/m_4112308_nw_10_060_20180926_20190209.tif
159 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/42123/m_4212361_sw_10_060_20181011_20190210.tif
160 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/42123/m_4212362_sw_10_060_20181011_20190210.tif
161 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/42123/m_4212360_sw_10_060_20181011_20190210.tif
162 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/42123/m_4212360_se_10_060_20181011_20190210.tif
163 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/201

226 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/42122/m_4212258_sw_10_060_20180926_20190209.tif
227 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/42122/m_4212260_se_10_060_20180925_20190209.tif
228 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/42122/m_4212263_sw_10_060_20180925_20190210.tif
229 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/42122/m_4212259_sw_10_060_20180925_20190209.tif
230 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/42122/m_4212261_sw_10_060_20180925_20190209.tif
231 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/42122/m_4212259_se_10_060_20180925_20190209.tif
232 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/42122/m_4212260_sw_10_060_20180925_20190209.tif
233 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/201

308 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/41121/m_4112130_sw_10_060_20181015_20190210.tif
309 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/41121/m_4112129_se_10_060_20181015_20190210.tif
310 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/41121/m_4112128_sw_10_060_20180924_20190209.tif
311 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/41121/m_4112129_sw_10_060_20181015_20190210.tif
312 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/41121/m_4112128_se_10_060_20180924_20190209.tif
313 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/41121/m_4112131_sw_10_060_20181015_20190210.tif
314 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/41121/m_4112132_sw_10_060_20181015_20190209.tif
315 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/201

...

4424 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/38120/m_3812051_sw_10_060_20180715_20190209.tif
4425 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/38120/m_3812050_se_10_060_20180715_20190209.tif
4426 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/38120/m_3812049_sw_10_060_20180714_20190209.tif
4427 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/38120/m_3812052_sw_10_060_20180725_20190210.tif
4428 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/38120/m_3812050_sw_10_060_20180715_20190209.tif
4429 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/38120/m_3812049_se_10_060_20180715_20190209.tif
4430 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/38120/m_3812052_ne_10_060_20180725_20190209.tif
4431 of 4781
https://naipeuwest.blob.core.windows.net/naip/v00

4526 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37119/m_3711905_se_11_060_20180831_20190209.tif
4527 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37119/m_3711901_sw_11_060_20180909_20190209.tif
4528 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37119/m_3711903_se_11_060_20180909_20190208.tif
4529 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37119/m_3711902_sw_11_060_20180909_20190208.tif
4530 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712008_se_10_060_20180909_20190210.tif
4531 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37119/m_3711903_sw_11_060_20180909_20190208.tif
4532 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37119/m_3711902_se_11_060_20180909_20190208.tif
4533 of 4781
https://naipeuwest.blob.core.windows.net/naip/v00

4608 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37118/m_3711801_se_11_060_20180831_20190208.tif
4609 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37119/m_3711906_sw_11_060_20180831_20190209.tif
4610 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37118/m_3711804_sw_11_060_20180908_20190209.tif
4611 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37119/m_3711907_se_11_060_20180831_20190209.tif
4612 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37119/m_3711907_sw_11_060_20180831_20190209.tif
4613 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37118/m_3711802_se_11_060_20180831_20190208.tif
4614 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37118/m_3711802_sw_11_060_20180831_20190208.tif
4615 of 4781
https://naipeuwest.blob.core.windows.net/naip/v00

4706 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712029_ne_10_060_20180815_20190209.tif
4707 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712029_nw_10_060_20180724_20190209.tif
4708 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712021_sw_10_060_20180724_20190210.tif
4709 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712022_se_10_060_20180815_20190209.tif
4710 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712021_se_10_060_20180815_20190210.tif
4711 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712020_se_10_060_20180725_20190210.tif
4712 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37120/m_3712022_sw_10_060_20180725_20190209.tif
4713 of 4781
https://naipeuwest.blob.core.windows.net/naip/v00

4772 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37118/m_3711826_ne_11_060_20180831_20190208.tif
Calculate NDVI
m_3711826_ne_11_060_20180831_20190208_ndvi.tif Complete
4773 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37118/m_3711826_nw_11_060_20180831_20190208.tif
Calculate NDVI
m_3711826_nw_11_060_20180831_20190208_ndvi.tif Complete
4774 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37118/m_3711825_ne_11_060_20180831_20190208.tif
Calculate NDVI
m_3711825_ne_11_060_20180831_20190208_ndvi.tif Complete
4775 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37118/m_3711827_nw_11_060_20180831_20190209.tif
Calculate NDVI
m_3711827_nw_11_060_20180831_20190209_ndvi.tif Complete
4776 of 4781
https://naipeuwest.blob.core.windows.net/naip/v002/ca/2018/ca_060cm_2018/37118/m_3711820_sw_11_060_20180908_20190208.tif
Calculate NDVI
m_3711820_sw_11_060_20180908_20